In [1]:
import sys
print(sys.path)

['/home/yichi/research/ad_afqmc', '/home/yichi/research/ad_afqmc/ad_afqmc', '/home/yichi/research/pyscf', '/home/yichi/research', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python311.zip', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11/lib-dynload', '', '/home/yichi/miniconda3/envs/lno_afqmc/lib/python3.11/site-packages']


In [2]:
#from functools import partial

#import numpy
from pyscf import gto, scf,cc
import os
from ad_afqmc import pyscf_interface, driver, mpi_jax,  run_afqmc
from ad_afqmc import config

# Hostname: YICHI
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [3]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print(size,rank)

1 0


In [1]:
from pyscf import gto, scf
#import os
from ad_afqmc import pyscf_interface, driver, mpi_jax,  run_afqmc
from ad_afqmc import config
atom = '''
 O   -1.485163346097   -0.114724564047    0.000000000000
 H   -1.868415346097    0.762298435953    0.000000000000
 H   -0.533833346097    0.040507435953    0.000000000000
'''

mol = gto.Mole(
    verbose=4,
    atom=atom,
    basis='ccpvdz',
    )

mol.build()
mf = scf.RHF(mol).density_fit()
mf.max_cycle = 5
mf.kernel()

moi,_,stable,_ = mf.stability(return_status=True)
if not stable: mf = mf.newton().run(moi, mf.mo_occ)
#else: print('HF RESULT IS STABLE')
    



# Hostname: YICHI
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
System: uname_result(system='Linux', node='YICHI', release='4.4.0-22621-Microsoft', version='#3672-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0]
numpy 1.24.0  scipy 1.14.1  h5py 3.11.0
Date: Mon Oct 14 10:12:49 2024
PySCF version 2.7.0
PySCF path  /home/yichi/research/pyscf
GIT HEAD (branch yichi) 410d9608a73ad3a507b3008a6f68549288ebfc33

[ENV] PYSCF_EXT_PATH /home/yichi/research/pyscf:
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O     -1.485163346097  -0.114724564047   0.00000

In [5]:
config.afqmc_config["use_gpu"]

False

In [4]:
# ad afqmc

pyscf_interface.prep_afqmc(mf)
options = {
    "dt":0.001,
    "n_eql": 1,
    "n_ene_blocks": 1,
    "n_sr_blocks": 50,
    "n_blocks": 1,
    "n_walkers": 50,
    "seed": 98,
    "trial": "rhf",
    "walker_type": "rhf",
    #"LNO":True,
    # "ad_mode": "reverse",
}
#pyscf_interface.prep_afqmc(mf)
#from mpi4py import MPI

#MPI.Finalize()
e_afqmc, err_afqmc = run_afqmc.run_afqmc_lno_mf(options=options)

#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (5, 5)
# Number of basis functions: 7
# Number of Cholesky vectors: 104
#
# Hostname: YICHI
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 7
# nelec: (5, 5)
#
# dt: 0.001
# n_eql: 2
# n_ene_blocks: 1
# n_sr_blocks: 50
# n_blocks: 3
# n_walkers: 50
# seed: 98
# trial: rhf
# walker_type: rhf
# n_prop_steps: 50
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# free_projection: False
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#
# Equilibration sweeps:
#   Iter        Block energy      Walltime
#     0      -7.567857497e+01     1.57e+00 
#     1      -7.572489929e+01     6.00e+00 
#     2      -7.572996521e+01     1.01e+01 
#
# Sampling sweeps:
#  Iter        Mean energy          Stochastic error       Walltime
     0      -

In [2]:
from lno.afqmc import LNOAFQMC
filename = 'fragmentenergies.txt'
for thresh in [1e-5]:
    f = open(filename,'w')
    f.close()
    mfcc = LNOAFQMC(mf, thresh=thresh, frozen=1).set(verbose=5)
    mfcc.thresh_occ = thresh*10
    mfcc.thresh_vir = thresh
    mfcc.nblocks = 1
    mfcc.nwalk_per_proc = 5#10
#    mfcc.seed = 1234 
#    mfcc.runfrags=[4]
    mfcc.lo_type = 'pm'
    mfcc.no_type = 'cim'
    mfcc.frag_lolist = '1o'
    mfcc.force_outcore_ao2mo = True
#    mfcc.vmc_root = vmc_root
    mfcc.kernel() #canonicalize=False,chol_vecs=chol_vecs)
    #ecc = mfcc.e_corr #+ mf.energy_nuc()
    #ecc_pt2corrected = mfcc.e_corr_pt2corrected(mmp.e_corr)
    #ecc_t = mfcc.e_corr_ccsd_t
#    log.info('thresh = %.0e  E_corr(AFQMC)     = %.10f, %.10f  rel(wrt CCSD) = %6.2f%%  '
#             'diff = % .10f',
#             thresh, ecc,mf.energy_nuc(), ecc/mcc.e_corr*100, ecc-mcc.e_corr)

    #log.info('thresh = %.0e  E_corr(AFQMC) = %.10f  E_corr(AFQMC+PT2) = %.10f', thresh, ecc, ecc_pt2corrected)


******** <class 'lno.afqmc.afqmc.LNOAFQMC'> ********
nocc = 4, nmo = 23
frozen orbitals 1
max_memory 4000 MB (current use 186 MB)
thresh_occ = 1.000000e-04
thresh_vir = 1.000000e-05
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 4  nlo = 4
frag_atmlist = None
frag_lolist = [[0], [1], [2], [3]]
frag_wghtlist = [1. 1. 1. 1.]
frag_nonvlist = None
    CPU time for LO and fragment       29.97 sec, wall time      2.97 sec
ao2mo est mem= 0.13 MB  avail mem= 3811.51 MB
Lov is saved to /tmp/6axqt0kw
    CPU time for transforming DF-MP2 integrals      0.00 sec, wall time      0.00 sec
    CPU time for Integral xform         0.02 sec, wall time      0.01 sec
    CPU time for make pno               0.00 sec, wall time      0.00 sec
Solving impurity problem
#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number o

In [8]:
options = {
    "dt":0.001,
    "n_eql": 1,
    "n_ene_blocks": 1,
    "n_sr_blocks": 50,
    "n_blocks": 1,
    "n_walkers": 50,
    "seed": 98,
    "trial": "rhf",
    "walker_type": "rhf",
    "LNO":True,
    # "ad_mode": "reverse",
}
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = mpi_jax._prep_afqmc(options)
driver.LNOafqmc(ham_data, ham, prop, trial, wave_data, sampler, observable, options)

# Number of MPI ranks: 1
#
# norb: 3
# nelec: (1, 1)
#
# dt: 0.001
# n_eql: 1
# n_ene_blocks: 1
# n_sr_blocks: 50
# n_blocks: 1
# n_walkers: 50
# seed: 98
# trial: rhf
# walker_type: rhf
# LNO: True
# n_prop_steps: 50
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# free_projection: False
# n_batch: 1
# orbE: 0
# maxError: 0.001
#
# Equilibration sweeps:
#   Iter        Block energy      Walltime
#     0      -7.567857491e+01     1.19e+00 


AttributeError: 'NoneType' object has no attribute 'T'